# Avaliação Final Tecnologias Hacker - Análise de Logs Web

## Objetivo
O objetivo desse projeto é construir um código capaz de analisar se, de acordo com os logs, pode haver uma conexão suspeita no seu servidor web. Para isso, levamos em consideração que os logs disponibilizados estão em formato combined.

In [1]:
#bibliotecas
import re

## Pré Processamento

In [2]:
#Função que le o arquivo e retorna uma lista com as linhas do arquivo
def ler_arquivo(nome_arquivo):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            return arquivo.readlines()
    except FileNotFoundError:
        print("Arquivo não encontrado")
        return None

In [3]:
lista_linhas_log = ler_arquivo('logs/access.log')
lista_linhas_log[0]

'207.46.13.104 - - [08/Jul/2019:09:07:18 +0200] "GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1" 302 623 "-" "Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"\n'

In [4]:
# Função de extração das informações
def extrai_info_log(lista_linhas):
    info_logs = {}
    for i in range(len(lista_linhas)):
        dic_log = {}
        info = lista_linhas[i].split()
        dic_log['ip'] = info[0]
        dic_log['id_cliente'] = info[1]
        dic_log['usuario'] = info[2]
        dic_log['data'] = info[3].replace('[','')
        dic_log['requisicao'] = info[5] + " " + info[6] + " " + info[7]
        dic_log['status'] = info[8]
        dic_log['bytes_resposta'] = info[9]
        dic_log['referenciador'] = info[10]
        dic_log['navegador'] = info[11].replace('"','')

        info_logs[i] = dic_log

    return info_logs




In [5]:
dicionariodelogs = extrai_info_log(lista_linhas_log)
dicionariodelogs[0]

{'ip': '207.46.13.104',
 'id_cliente': '-',
 'usuario': '-',
 'data': '08/Jul/2019:09:07:18',
 'requisicao': '"GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1"',
 'status': '302',
 'bytes_resposta': '623',
 'referenciador': '"-"',
 'navegador': 'Mozilla/5.0'}

In [6]:
#Função para separa somente os logs de get
def separa_logs_get(dicionario_logs):
    logs_get = {}
    for i in range(len(dicionario_logs)):
        if 'GET' in dicionario_logs[i]['requisicao']:
            logs_get[i] = dicionario_logs[i]
    return logs_get

#Função que separa os logs de post
def separa_logs_post(dicionario_logs):
    logs_post = {}
    for i in range(len(dicionario_logs)):
        if 'POST' in dicionario_logs[i]['requisicao']:
            logs_post[i] = dicionario_logs[i]
    return logs_post

In [7]:
dicionariodelogs_get = separa_logs_get(dicionariodelogs)
dicionariodelogs_post = separa_logs_post(dicionariodelogs)

### Analisando os logs contendo POST
Muitas vezes, pessoas má influenciadas tentam "injetar" no servidor muitas vezes isso acontece por meio de um "POST".

In [ ]:
def analisando_os_posts(log_post):
    logs_suspeitos = {}
    padroes_suspeitos = ['wp-login', 'wp-admin', 'xmlrpc.php', 'admin-ajax.php']

    for key, value in log_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao']:
                logs_suspeitos[key] = value
                break 
    return logs_suspeitos


In [25]:
logs_post_suspeitos = analisando_os_posts(dicionariodelogs_post)
logs_post_suspeitos

{1111: {'ip': '178.128.116.50',
  'id_cliente': '-',
  'usuario': '-',
  'data': '11/Jul/2019:00:52:50',
  'requisicao': '"POST /wp-admin/admin-ajax.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '526',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 24062: {'ip': '89.108.87.179',
  'id_cliente': '-',
  'usuario': '-',
  'data': '22/Aug/2019:06:36:02',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '500',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 26426: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /blog/xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '547',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 26427: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '537',
  'referenciador'